downgrade tensorflow 2.2.0 -> 1.14.0

In [ ]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 37kB/s 
     |████████████████████████████████| 3.2MB 49.8MB/s 
     |████████████████████████████████| 491kB 44.9MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  6 13:54:17 2020

@author: Taehan Kim
"""




import glob #file directory
import shutil
import os
import random
from PIL import Image
import seaborn as sns #statistical data visualization
import pandas as pd
import pandas.util.testing as tm
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K




/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, 

Google Drive에 마운트


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#%% mask_df는 정답 마스크에 대해 정의 하고 이를 정리해놓은 변수
mask_filenames = glob.glob('/content/gdrive/My Drive/Colab Notebooks/masks/*')
mask_df = pd.DataFrame()
mask_df['filename'] = mask_filenames
mask_df['mask_percentage'] = 0
mask_df['labels'] = 0
mask_df.set_index('filename', inplace=True)

for file in mask_filenames:
    mask_df.loc[file, 'mask_percentage'] = np.array(Image.open(file)).sum()/(128*128)

mask_df.loc[mask_df.mask_percentage > 0, 'labels'] = 1
#%% 
train_valid_filenames = glob.glob('/content/gdrive/My Drive/Colab Notebooks/train/*')
#전부 마스크에 대한 변수가 됩니다. 
train_filenames, valid_filenames = train_test_split(train_valid_filenames, stratify = mask_df.labels, test_size = 0.2, random_state = 10)
mask_train_filenames = [f.replace('/train', '/masks') for f in train_filenames]
mask_valid_filenames = [f.replace('/train', '/masks') for f in valid_filenames]
# from efficientnet import EfficientNetB0

In [ ]:
#%%
train_x = np.zeros((len(train_filenames),128,128))
valid_x = np.zeros((len(valid_filenames),128,128))
train_mask_y = np.zeros((len(train_filenames),128,128))
valid_mask_y = np.zeros((len(valid_filenames),128,128))
train_y = np.zeros((len(train_filenames)))
valid_y = np.zeros((len(valid_filenames)))

# 훈련용 .train image
for (index, image) in enumerate(train_filenames[:]): #index와 내용을 함께
    train_x[index] = np.array(Image.open(image))
    
# 검증용 및 변병여부 .train image
for (index, image) in enumerate(valid_filenames[:]):
    valid_x[index] = np.array(Image.open(image))
    
# 훈련용 마스크 답지 및 병변여부.mask image
for (index, image) in enumerate(mask_train_filenames[:]):
    train_mask_y[index] = np.array(Image.open(image))
    train_y[index] = mask_df.loc[image, 'labels']
    
# 검증용 병변 답지 및 병변여부 .mask image
for (index, image) in enumerate(mask_valid_filenames[:]):
    valid_mask_y[index] = np.array(Image.open(image))
    valid_y[index] = mask_df.loc[image, 'labels']

In [ ]:
train_x = train_x.reshape(len(train_filenames),128,128,1)
valid_x = valid_x.reshape(len(valid_filenames),128,128,1)

train_mask_y = train_x.reshape(len(train_filenames),128,128,1)
valid_mask_y = valid_x.reshape(len(valid_filenames),128,128,1)

In [ ]:
def UNet(pretrained_weights = None,input_size = (128,128,1)):
    inp = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.2)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.2)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inp, outputs=[conv10])

    return model

In [ ]:
model = UNet()
model.summary()
model.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 128, 128, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 128, 128, 64) 36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 64, 64, 64)   0           conv2d_25[0][0]                  
____________________________________________________________________________________________

In [ ]:
history = model.fit(train_x/65535, [train_mask_y/65535],
                                    epochs = 10, 
                                    batch_size = 256,
                                    verbose = 1, 
                                    validation_data=(valid_x/65535,[valid_mask_y/65535]))

Train on 2252 samples, validate on 564 samples
Epoch 1/10
2252/2252 [==============================] - 2313s 1s/sample - loss: 7.5917 - acc: 0.0140 - val_loss: 0.6600 - val_acc: 0.0134
Epoch 2/10
2252/2252 [==============================] - 2345s 1s/sample - loss: 0.6092 - acc: 0.0140 - val_loss: 0.5265 - val_acc: 0.0134
Epoch 3/10
2252/2252 [==============================] - 2331s 1s/sample - loss: 0.4863 - acc: 0.0140 - val_loss: 0.4093 - val_acc: 0.0134
Epoch 4/10
2252/2252 [==============================] - 2344s 1s/sample - loss: 0.4118 - acc: 0.0140 - val_loss: 0.4123 - val_acc: 0.0134
Epoch 5/10
2252/2252 [==============================] - 2337s 1s/sample - loss: 0.4010 - acc: 0.0140 - val_loss: 0.3961 - val_acc: 0.0134
Epoch 6/10
2252/2252 [==============================] - 2355s 1s/sample - loss: 0.3935 - acc: 0.0140 - val_loss: 0.3940 - val_acc: 0.0134
Epoch 7/10
2252/2252 [==============================] - 2361s 1s/sample - loss: 0.3902 - acc: 0.0140 - val_loss: 0.3924 - val

KeyboardInterrupt: ignored

Accuracy & Loss visualization

In [ ]:
from matplotlib import pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
 
